# DART 고유번호 수집

- [다트 고유번호](https://opendart.fss.or.kr/guide/detail.do?apiGrpCd=DS001&apiId=2019018)

In [2]:
### 필요한 모듈
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import pandas as pd

from time import sleep
import requests


In [6]:
api_key = "801b9fc87e380dfc40a19a593d6fd762bc681388"

api와 url을 통해 다트에 등록된 고유번호를 추출한다.

In [8]:

### 회사고유번호 데이터 불러오기
url = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key=' + api_key
with urlopen(url) as ziprep:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('corp_num')

이렇게 고유 xml 파일을 corp_num 폴더에 수집하게 되었다. xml파일 이름은 CORPCODE.xml 이다.

## 고유번호 변환

ElementTree 모듈을 활용해서 corp_num/CORPCODE.xml을 파싱한다. 루트를 가져와서 하나하나 판다스 데이터 프레임으로 변환한다.

In [9]:
### 압축파일 안의 xml 파일 읽기
tree = ET.parse('corp_num/CORPCODE.xml')
root = tree.getroot()

이제 본격적으로 데이터 프레임으로 바꿔주겠다. 루트의 '리스트'라는 이름을 반복문으로 사용하여 하나하나씩 데이터 프레임으로 만들어주고, 이것을 pd.concat()를 사용하여 하나씩 이어붙이도록 하겠다.

In [13]:
corp_info_df = pd.DataFrame() 
corp_info_df

""


In [14]:

for country in root.iter("list"):
    country_dic = [{
        "corp_code":country.findtext("corp_code"),
        "corp_name":country.findtext("corp_name"),
        "stock_code":country.findtext("stock_code"),
        "modify_date":country.findtext("modify_date")
    }]
    
    df = pd.DataFrame(country_dic)
    corp_info_df = pd.concat([corp_info_df,df])

이렇게 수집된 corp_info 데이터는 corp_info_raw.csv로 저장하겠다.

In [16]:
# corp_info_df.to_csv("corp_info_raw.csv", index = False)

### 데이터 로드

이렇게 수집한 corp_info_raw.csv를 한번 불러와보자. corp_code는 앞에 0이 있기 때문에, 불러올때 문자열로 변환을 시켜주자. 이때 사용하는 옵션은 dtype로 corp_code를 str로 변환시켜주는 역할을 한다.

In [10]:
corp_info_df = pd.read_csv("corp_info_raw.csv", dtype = {'corp_code': str})

In [11]:
corp_info_df

,corp_code,corp_name,stock_code,modify_date
0,00434003,다코,,20170630
1,00434456,일산약품,,20170630
2,00430964,굿앤엘에스,,20170630
3,00432403,한라판지,,20170630
4,00388953,크레디피아제이십오차유동화전문회사,,20170630
...,...,...,...,...
98028,00646510,미래에셋파트너스사호사모투자전문회사,,20230228
98029,01184822,미래에셋파트너스9호사모투자,,20230228
98030,00755252,시니안,,20230228
98031,00227582,청호나이스,,20230228


상장된 회사들만 추려보자.

lambda를 사용해서 문자열이 6인것만 추려서 stock code를 가져오자.

In [14]:
corp_info_df = corp_info_df[corp_info_df.stock_code.apply(lambda x: len(str(x))==6)]
corp_info_df = corp_info_df.reset_index(drop=True)

In [15]:
corp_info_df

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
...,...,...,...,...
3560,00413417,우리손에프앤지,073560,20230403
3561,00440712,어반리튬,073570,20230403
3562,00483735,해성옵틱스,076610,20230403
3563,00516246,알에프세미,096610,20230403


이렇게 우리는 주가 데이터만 뽑아서 수집을 해보았다. 그러나 '한빛네트'라는 회사는 부도가 난 회사이므로 실제로 주가데이터에 영향을 주지 않는다. 다음에는 이 데이터를 추려놓고 작업을 하는 쪽으로 하고 이번에는 이 데이터를 저장하고 마무리해보자.

In [16]:
corp_info_df.to_csv("corp_info_stock_code.csv", index = False)

In [15]:
dfdf = pd.read_csv("corp_info_stock_code.csv")

In [16]:
dfdf['corp_code'] = dfdf['corp_code'].astype(str).str.zfill(8)
dfdf['stock_code'] = dfdf['stock_code'].astype(str).str.zfill(6)

In [17]:

dfdf

,corp_code,corp_name,stock_code,modify_date
0,00260985,한빛네트,036720,20170630
1,00264529,엔플렉스,040130,20170630
2,00358545,동서정보기술,055000,20170630
3,00231567,애드모바일,032600,20170630
4,00247939,씨모스,037600,20170630
...,...,...,...,...
3560,00413417,우리손에프앤지,073560,20230403
3561,00440712,어반리튬,073570,20230403
3562,00483735,해성옵틱스,076610,20230403
3563,00516246,알에프세미,096610,20230403
